In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
import random
import pickle
from sklearn.preprocessing import StandardScaler
import os
import tensorflow as tf
from tqdm.notebook import tqdm
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

2023-06-07 11:56:24.851199: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-07 11:56:24.854709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 11:56:24.892139: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 11:56:24.894117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 11:56:25.767910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
def get_targets_with_weights(batch_data, batch_data1, initial_ensembles, size_ens): 

    weights_ann_1 = ann_15.get_weights()
    weights_ann_2 = ann_20.get_weights()
    
    n_hidden_1 = len(weights_ann_1[0].ravel())
    
    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)
    
    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)


    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, target_dim)


    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + target_dim)].reshape(size_ens, 1, target_dim)


    final_output_1 = output_1 + output_layer_bias_1



    n_hidden_2 = len(weights_ann_2[0].ravel())

    initial_ensembles_1 = initial_ensembles.copy()[:, total_weights_1:(total_weights_1+ total_weights_2)]

    hidden_weights_2 = initial_ensembles_1[:,:n_hidden_2].reshape(size_ens, batch_data1.shape[1], h2)



    hidden_output_2 = np.einsum('ij,kjl->kil', batch_data1, hidden_weights_2)

    hidden_layer_bias_2 = initial_ensembles_1[:,n_hidden_2:(n_hidden_2 + h2)].reshape(size_ens, 1,  h2)


    # hidden_layer_bias_2 = np.expand_dims(hidden_layer_bias_2, 0)

    hidden_output_2 = hidden_output_2+ hidden_layer_bias_2
#     hidden_layer_bias_2 = hidden_layer_bias_2 + hidden_layer_bias_2


    n_pred_weights_2 = len(weights_ann_2[2].ravel())

    output_weights_2 = initial_ensembles_1[:,(n_hidden_2 + h2):(n_hidden_2 + h2 + n_pred_weights_2) ].reshape(size_ens, h2, target_dim)


    output_2 = np.einsum('ijk,ikl->ijl', hidden_output_2, output_weights_2)


    output_layer_bias_2 = initial_ensembles_1[:,(n_hidden_2 + h2 + n_pred_weights_2):(n_hidden_2 + h2 + n_pred_weights_2 + target_dim)].reshape(size_ens, 1, target_dim)

    # output_layer_bias_2 = np.expand_dims(output_layer_bias_2, 0)


    final_output_2 = output_2 + output_layer_bias_2


    weights_1 = initial_ensembles[:, :total_weights_1]

    weights_2 = initial_ensembles[:, total_weights_1:(total_weights_1 + total_weights_2)]

#     print(weights_1.shape)
    
    # print(weights_2.shape)

    avg_weights = initial_ensembles[:, -4].reshape(-1,1)
    
    cov_matrix_parms =  initial_ensembles[:, -3:].reshape(-1,1)
    
    # avg_weights_not_sig = avg_weights.reshape(avg_weights.shape[0], 1, avg_weights.shape[1])
    
    
    avg_weights_sig = expit(avg_weights)
    
    avg_weights_sig = avg_weights_sig.reshape(avg_weights_sig.shape[0], 1, avg_weights_sig.shape[1])
    
    
    complement_weights_sig = 1 - expit(avg_weights)
    
    complement_weights_sig = complement_weights_sig.reshape(complement_weights_sig.shape[0], 1, complement_weights_sig.shape[1])
    

    final_output_1 = final_output_1*complement_weights_sig
    
    final_output_2 = final_output_2*avg_weights_sig
    
    return final_output_1 + final_output_2

In [3]:
def ann(hidden = 32, input_shape = 256, output_shape = 2): 
    input_layer = tf.keras.layers.Input(shape = (input_shape))
    hidden_layer = tf.keras.layers.Dense(hidden)
    hidden_output = hidden_layer(input_layer)
    pred_layer = tf.keras.layers.Dense(output_shape, activation = "relu")
    pred_output = pred_layer(hidden_output)
#     pred_output = tf.keras.layers.Activation("softmax")(pred_output)
    model = tf.keras.models.Model(input_layer, pred_output)
    return model

In [4]:
def expit(x):
    """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
    return 1 / (1 + np.exp(-x))

In [5]:
alogp_bottleneck = np.load("..//Data/small_mol_phase_3_features_for_both.npy")

In [6]:
# y_valid

In [7]:
y_valid = pd.read_csv("..//Data/smiles_with_rdkit_with_small_phase_3_outputs.csv")

In [8]:
# y_valid

In [9]:
std_targets = pickle.load( open('..//Data//target_scaler.pkl', 'rb'))

/home/statgrads/vpiyush2/.conda/envs/enkf/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
# y_valid

In [11]:
y_train = y_valid.values[:,1:]

In [12]:
y_train = std_targets.transform(y_train)

/home/statgrads/vpiyush2/.conda/envs/enkf/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [13]:
# R_t = np.cov(y_train.T)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_valid, y_train, y_valid = train_test_split(alogp_bottleneck, y_train, test_size = 0.25, shuffle = True, 
                                                     random_state = 42)

In [16]:
x_train.shape

(719, 64)

In [17]:
from scipy.linalg import block_diag

In [18]:
# n = x_train.shape[0]
# R_t = block_diag(*([R_t] * n))

In [19]:
# R_t.shape

In [20]:
smiles_feats_train = x_train[:, :32]

In [21]:
rdkit_feats_train = x_train[:, 32:]

In [22]:
smiles_feats_valid = x_valid[:, :32]
rdkit_feats_valid = x_valid[:, 32:]

In [23]:
h1, h2 = 16, 16

In [24]:
ann_15 = ann(h1, 32, 2)

2023-06-07 11:56:27.005584: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-07 11:56:27.005610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: c2127.swan.hcc.unl.edu
2023-06-07 11:56:27.005615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: c2127.swan.hcc.unl.edu
2023-06-07 11:56:27.005680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.85.5
2023-06-07 11:56:27.005693: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.85.5
2023-06-07 11:56:27.005696: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.85.5


In [25]:
ann_20 = ann(h2, 32, 2)

In [26]:
total_weights_1 = ann_15.count_params()

In [27]:
total_weights_2 = ann_20.count_params()

In [28]:
total_weights = total_weights_1 + total_weights_2 + 1 + 3 + 2

In [29]:
total_weights

1130

In [30]:
# generate MVN from 0 and some var
initial_ensembles_mean = np.zeros((total_weights,))

In [31]:
lambda1 = 1

In [32]:
initial_ensembles_cov = lambda1*np.identity(total_weights)

In [33]:
reduction = 4

In [34]:
size_ens = total_weights//reduction

In [35]:
initial_ensembles = mvn(initial_ensembles_mean, initial_ensembles_cov).rvs(size_ens)

In [36]:
initial_ensembles.shape

(282, 1130)

In [37]:
# initial_ensembles[0, -5:-2]

In [38]:
def create_cov(shape, initial_ensembles, idx):
    cov_part = initial_ensembles[idx, -5:-2]
    # variances = tf.math.softplus(cov_part[:2]).numpy()
    variances = cov_part[:2]
    covariances = cov_part[-1]
    base_cov = np.identity(2)
    base_cov[0,0] = variances[0]
    base_cov[1,1] = variances[1]
    base_cov[0,1] = covariances
    base_cov[1,0] = covariances
    
    variances = tf.math.softplus(initial_ensembles[idx, -2:]).numpy()
    base_variances = np.identity(2)
    base_variances[0,0] = variances[0]
    base_variances[1,1] = variances[1]
    
    final = np.linalg.cholesky(base_cov@base_cov.T + base_variances)
    cov_mat = final@final.T
    cov_mat_final = cov_mat@cov_mat
    
    n = shape
    R_t = block_diag(*([cov_mat_final] * n))
    
    lambda_inv = np.linalg.inv(R_t)
    
    return lambda_inv
    

In [39]:
# np.linalg.det(np.array(all_covs))

In [41]:
# tf.math.softplus(-0.2043372)

In [42]:
# initial_ensembles[:, -3:-1]

In [43]:
# all_covs = np.array(all_covs)

In [56]:
# all_covs.shape

In [57]:
def calculate_mu_bar_G_bar(data1, data2, initial_ensembles):
    mu_bar = initial_ensembles.mean(0)
    G_u = get_targets_with_weights(data1, data2, initial_ensembles, size_ens = size_ens)
    G_u = G_u.reshape(G_u.shape[0], G_u.shape[1]*G_u.shape[2])
    G_bar = (G_u.mean(0)).ravel()
    return mu_bar.reshape(-1,1), G_bar.reshape(-1,1), G_u

In [58]:
def calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u): 
    u_j_minus_u_bar = initial_ensembles - mu_bar.reshape(1,-1)
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    c = np.zeros((total_weights, G_bar.shape[0]))
    for i in range(0, size_ens): 
        c += np.kron(u_j_minus_u_bar[i, :].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    return c/size_ens, G_u_minus_G_bar

In [59]:
def calculate_D_u( G_bar, G_u): 
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    d = np.zeros((G_bar.shape[0], G_bar.shape[0]))
    for i in range(0, size_ens): 
        d += np.kron(G_u_minus_G_bar[i,:].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    return d/size_ens

In [60]:
def get_updated_ensemble(data1, data2, initial_ensembles):
    mu_bar, G_bar, G_u = calculate_mu_bar_G_bar(smiles_feats_train, rdkit_feats_train, initial_ensembles)
    C, G_u_minus_G_bar = calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u)
    D = calculate_D_u( G_bar, G_u)
    all_covs = Parallel(n_jobs = 15, verbose = 0)(delayed(create_cov)(data1.shape[0],initial_ensembles, idx) for idx in range(size_ens))
    all_covs = np.array(all_covs)
    D_plus_cov = (D + all_covs)
    D_plus_cov_inv = np.linalg.inv(D_plus_cov)
    mid_quant = C@D_plus_cov_inv
    # mid_quant = C@np.linalg.inv(D + lambda_inv)
    right_quant = y_train.ravel().reshape(-1,1) - G_u.T
    mid_times_right = mid_quant@right_quant
    updated_ensemble = (initial_ensembles.T + mid_times_right)
    updated_ensemble = updated_ensemble.mean(0)
    return updated_ensemble.T

In [61]:
target_dim = 2

In [62]:
lambda_D = 1

In [63]:
u_bar, G_bar, G_u = calculate_mu_bar_G_bar(smiles_feats_train, rdkit_feats_train, initial_ensembles)

In [64]:
G_bar.shape

(1438, 1)

In [65]:
def inverse_transform(data, idx):
    data_cur = data[idx, :, :]
    inv_data_cur = std_targets.inverse_transform(data_cur)
    return inv_data_cur

In [ ]:
# catch = Parallel(n_jobs = 15, verbose = 3)(delayed(inverse_transform)(G_u_test, i)  for i in range(G_u_test.shape[0]))

In [66]:
for i in range(0,10000):
    
    initial_ensembles = get_updated_ensemble(smiles_feats_train, rdkit_feats_train, initial_ensembles)
    
    G_u_train = get_targets_with_weights(smiles_feats_train, rdkit_feats_train, initial_ensembles, size_ens = size_ens)
    catch = Parallel(n_jobs = 15, verbose = 0)(delayed(inverse_transform)(G_u_train, i)  for i in range(G_u_train.shape[0]))
    G_u_train = np.array(catch)
    
    y_train_cur = std_targets.inverse_transform(y_train)
    
    li_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[0,:,:]   
    ui_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[1,:,:]
    
    width_train = ui_train - li_train
    avg_width_train = width_train.mean(0)
    
    ind_train = (y_train_cur >= li_train) & (y_train_cur <= ui_train)
    coverage_train= ind_train.mean(0)
    
    averaged_targets_train = G_u_train.mean(0)
    rmse_train = np.sqrt(((y_train_cur -averaged_targets_train)**2).mean(0))
    print(rmse_train, coverage_train, avg_width_train)
    
    G_u_test = get_targets_with_weights(smiles_feats_valid, rdkit_feats_valid, initial_ensembles, size_ens = size_ens)
    
    catch = Parallel(n_jobs = 15, verbose = 0)(delayed(inverse_transform)(G_u_test, i)  for i in range(G_u_test.shape[0]))
    G_u_test = np.array(catch)
    
    y_valid_cur = std_targets.inverse_transform(y_valid)    
    
    li_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[0,:,:]   
    ui_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[1,:,:]
    
    width_test = ui_test - li_test
    avg_width_test = width_test.mean(0)
    
    ind_test = (y_valid_cur >= li_test) & (y_valid_cur <= ui_test)
    coverage_test= ind_test.mean(0)
    
    averaged_targets_test = G_u_test.mean(0)
    rmse_test = np.sqrt(((y_valid_cur -averaged_targets_test)**2).mean(0))    
    
    # plt.scatter(y_valid[:, 0], averaged_targets_test[:,0])
    # plt.axline((0,0), slope = 1, c= "black")
    # plt.show()
    # plt.scatter(y_valid[:,1], averaged_targets_test[:, 1])
    # plt.axline((0,0), slope = 1, c= "black")
    # plt.show()
    
    if coverage_train.mean() < 0.95:
        break
    
    print(rmse_test, coverage_test, avg_width_test)

[  2.12247029 116.83346494] [1. 1.] [ 100.2427185  2285.94471715]
[  2.00366785 103.11753901] [1. 1.] [  98.4813168  2270.17568915]
[ 2.06990245 74.54409833] [1. 1.] [  68.57682765 1562.86566629]
[ 2.10250172 69.49683215] [1. 1.] [  68.20513281 1548.27782088]
[ 1.83806057 58.94566369] [1. 1.] [  55.27521812 1255.38322481]
[ 1.82686523 54.91518436] [1. 1.] [  54.71043213 1249.53001387]
[ 1.87412405 66.84643257] [1. 1.] [  48.27196587 1121.66816016]
[ 1.8302688  61.81765731] [1. 1.] [  48.049355   1116.74301632]
[ 2.15255896 61.15247827] [1. 1.] [ 42.85634852 950.18270847]
[ 2.12896543 56.86872526] [1. 1.] [ 42.57586462 937.55193879]
[ 2.13739013 58.74039793] [1. 1.] [ 39.30210972 874.10011402]
[ 2.1234225  54.60104055] [1. 1.] [ 38.70131786 866.52885702]
[ 2.13685021 56.35677419] [1. 1.] [ 35.04597311 806.87831259]
[ 2.12027654 52.41927041] [1. 1.] [ 34.53077507 795.31045446]
[ 2.04235616 53.66420587] [1. 1.] [ 32.26104244 730.03689182]
[ 2.04859937 49.76434662] [1. 1.] [ 31.9029419  72

KeyboardInterrupt: 

In [ ]:
G_u_test.shape

In [ ]:
import random

In [ ]:
random_idx = random.sample(range(y_valid_cur.shape[0]), k = 8)

In [ ]:
fig, axs = plt.subplots(8, 2,figsize=(15, 15))
# axs = axs.ravel()
# counter = 0
for idx, i in enumerate(random_idx):
    # print(counter)
    truth = y_valid_cur[i,:]
    preds = G_u_test[:, i,:]
    percts = np.percentile(preds, axis = 0, q = (2.5, 97.5))
    lis = percts[0,:]
    uis = percts[1,:]
    
    
    axs[idx, 0].hist(preds[:,0])
    axs[idx, 0].axvline(truth[0], color='green', linewidth=2)
    axs[idx, 0].axvline(lis[0], color='red', linewidth=2)
    axs[idx, 0].axvline(uis[0], color='red', linewidth=2)
    
    axs[idx, 1].hist(preds[:,1])
    axs[idx, 1].axvline(truth[1], color='green', linewidth=2)
    axs[idx, 1].axvline(lis[1], color='red', linewidth=2)
    axs[idx, 1].axvline(uis[1], color='red', linewidth=2)
    
    # counter+=2
    # print(counter)
    
    # plt.show()
plt.savefig('prediction_intervals.png', bbox_inches='tight')

In [ ]:
plt.scatter(y_valid_cur[:, 0], averaged_targets_test[:,0])
plt.axline((0,0), slope = 1, c= "black")
plt.show()
plt.scatter(y_valid_cur[:,1], averaged_targets_test[:, 1])
plt.axline((0,0), slope = 1, c= "black")
plt.show()